In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import time
import pandas as pd
from helper import get_penmap

import warnings
warnings.simplefilter('ignore')

Using TensorFlow backend.


In [2]:
def compute_dist(param1,param2):
    d = 0
    for i in range(0,len(param1)):
        d+=np.nan_to_num(((param1[i]-param2[i])**2)/(param1[i]+param2[i]))
    return d

def dist(param1,param2,n1,n2):
    mean1 = np.mean(param1)
    mean2 = np.mean(param2)
    var1  = np.var(param1)
    var2 = np.var(param2)
     
    d = np.abs(mean1-mean2)
    d/=(var1/n1 + var2/n2)**0.5
    
    return d

def distance_distribution(features):
    same_pens = {}
    diff_pens = {}
    
    for k in range(1,113) :
        for i in ['1','2']: 
            for n in ['1','2']:
                for j in ['1','2','3','4','5','6','7','8'] :
                    for m in ['1','2','3','4','5','6','7','8'] :
                        try:
                            if(i==n and j==m):
                                continue    
                            elif i==n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in same_pens:
                                same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m])
                    
                            elif i!=n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in diff_pens :                               
                                diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m])
                        except:
                            
                            pass
     
    
    same_pens = [ v for v in same_pens.values() ]
    diff_pens = [ v for v in diff_pens.values() ]
    return same_pens,diff_pens

def pen_distribution(features,a,pen_out):
    same_pens = {}
    diff_pens = {}
    
    for k in range(1,113) :
        if(k in a[pen_out]):
            for i in ['1','2']: 
                for n in ['1','2']:
                    for j in ['1','2','3','4','5','6','7','8'] :
                        for m in ['1','2','3','4','5','6','7','8'] :
                            try:
                                if(i==n and j==m):
                                    continue    
                                elif i==n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in same_pens:
                                    same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m])

                                elif i!=n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in diff_pens :                               
                                    diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m])
                            except:

                                pass

    
    same_pens = [ v for v in same_pens.values() ]
    diff_pens = [ v for v in diff_pens.values() ]
    return same_pens,diff_pens

In [3]:
def distance_btw_distributions(feature,a):
    d = []
    same_pens,diff_pens = distance_distribution(feature)
    D = dist(same_pens,diff_pens,1465,1456)
    for j in range(1,15):
        same_pens_i,diff_pens_i= pen_distribution(feature,a,j)
        d1 = len(same_pens_i)/2930*dist(same_pens_i,same_pens,len(same_pens_i),1465)
        d2 = len(diff_pens_i)/2912*dist(diff_pens_i,diff_pens,len(diff_pens_i),1456)
        d.append(d1+d2)
        
    return D-sum(d)

In [4]:
def load_parameters(path):
    hist = {}
    a = pd.read_csv(path)
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    
                    hist['c'+str(k)+'_'+i+'_'+j] = a['c'+str(k)+'_'+i+'_'+j]
                    hist['c'+str(k)+'_'+i+'_'+j].reset_index(inplace=True, drop=True)

                    
                except:
                    break
    return hist

In [5]:
def concat2(h1,h2):
    hist = {}    
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    hist['c'+str(k)+'_'+i+'_'+j] = np.concatenate([h1['c'+str(k)+'_'+i+'_'+j],h2['c'+str(k)+'_'+i+'_'+j]])
                except:
                    break
    return hist

def concat3(h1,h2,h3):
    hist = {}    
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    hist['c'+str(k)+'_'+i+'_'+j] = np.concatenate([h1['c'+str(k)+'_'+i+'_'+j],h2['c'+str(k)+'_'+i+'_'+j],h3['c'+str(k)+'_'+i+'_'+j]])
                except:
                    break
    return hist

def concat4(h1,h2,h3,h4):
    hist = {}    
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    hist['c'+str(k)+'_'+i+'_'+j] = np.concatenate([h1['c'+str(k)+'_'+i+'_'+j],h2['c'+str(k)+'_'+i+'_'+j],h3['c'+str(k)+'_'+i+'_'+j],h4['c'+str(k)+'_'+i+'_'+j]])
                except:
                    break
    return hist

In [6]:
a = get_penmap()

In [7]:
dir = '/home/mohit/Desktop/featuresUpdated/'
path1 = 'LBP_selected_features.csv'
path2 = 'Glcm_Selectedfeatures.csv'
path3 = 'statistical_Selectedfeatures.csv'
path4 = 'Gabor_Selectedfeatures.csv'

In [8]:
fe = {}
fe[1] = load_parameters(dir+path1)
fe[2] = load_parameters(dir+path2)
fe[3] = load_parameters(dir+path3)
fe[4] = load_parameters(dir+path4)

In [9]:

# min max normalising every feature except LBP 

for i in range(0,24):
    mi = np.min([ v[i] for v in fe[2].values() ])
    mx = np.max([ v[i] for v in fe[2].values() ])
    for key in fe[2].keys():
        fe[2][key][i]-=mi
        fe[2][key][i]/=(mx-mi)  

for i in range(0,3):
    mi = np.min([ v[i] for v in fe[3].values() ])
    mx = np.max([ v[i] for v in fe[3].values() ])
    for key in fe[1].keys():
        fe[3][key][i]-=mi
        fe[3][key][i]/=(mx-mi)  
        
for i in range(0,24):
    mi = np.min([ v[i] for v in fe[4].values() ])
    mx = np.max([ v[i] for v in fe[4].values() ])
    for key in fe[4].keys():
        fe[4][key][i]-=mi
        fe[4][key][i]/=(mx-mi)  

In [11]:
fe[1]['c1_1_1'].shape,fe[2]['c1_1_1'].shape,fe[3]['c1_1_1'].shape,fe[4]['c1_1_1'].shape

((76,), (24,), (3,), (24,))

In [12]:
Distances = {}

for i in range(1,5):
    Distances[i] = distance_btw_distributions(fe[i],a)
        
sorted_by_value = sorted(Distances.items(), key=lambda kv: kv[1],reverse =True)
sorted_by_value

[(2, 29.376019119989756),
 (1, 27.664684340890375),
 (3, 26.047017789216387),
 (4, 23.916760255033072)]

In [13]:
fmax = [2]
dmax = [29.37601911998976]
Fmax = fe[2]
Dmax =sum(dmax)
f = range(1,5)

In [14]:
while(set(f)-set(fmax)):
    Distances1 = {}
    for i in set(f)-set(fmax):
        Distances1[i] = distance_btw_distributions(concat2(Fmax,fe[i]),a)-Dmax
        
    sorted_by_value = sorted(Distances1.items(), key=lambda kv: kv[1],reverse =True)
    print("Stepsss: {}".format(sorted_by_value))
    dd = sorted_by_value[0][1]
    index = sorted_by_value[0][0]
    
    print(index,dd)
    if(dd<0):
        break
                
    fmax.append(index)
    dmax.append(dd)
    Dmax = sum(dmax)
    Fmax = concat2(fe[index],Fmax)

Stepsss: [(4, 2.2128821435673736), (1, 2.0987035994671537), (3, 1.1990140284602653)]
4 2.2128821435673736
Stepsss: [(1, 0.9116107890660388), (3, 0.602944896488701)]
1 0.9116107890660388
Stepsss: [(3, 0.43223299319396347)]
3 0.43223299319396347


In [15]:
fmax,dmax,Dmax

([2, 4, 1, 3],
 [29.37601911998976,
  2.2128821435673736,
  0.9116107890660388,
  0.43223299319396347],
 32.932745045817136)

# So ALL features are selected !!

In [18]:
best = pd.DataFrame(concat2(concat3(load_parameters(dir+path2),load_parameters(dir+path3),load_parameters(dir+path4)),fe[1]))
best

,c100_1_1,c100_1_2,c100_1_3,c100_1_4,c100_2_1,c100_2_2,c100_2_3,c101_1_1,c101_1_2,c101_1_3,...,c99_2_3,c9_1_1,c9_1_2,c9_1_3,c9_1_4,c9_1_5,c9_1_6,c9_1_7,c9_2_1,c9_2_2
0,0.272580,0.343392,0.374409,0.304421,0.214561,0.249565,0.249783,0.344433,0.342782,0.297526,...,0.151027,0.387566,0.403251,0.499124,0.508989,0.500553,0.411988,0.501667,0.492740,0.424042
1,0.712217,0.734085,0.700457,0.697822,0.164977,0.150165,0.131484,0.310955,0.308627,0.239849,...,0.163037,0.695568,0.728904,0.686154,0.696058,0.667423,0.701632,0.678424,0.763586,0.789137
2,0.298065,0.308474,0.312066,0.303538,0.632220,0.607482,0.593275,0.161111,0.164981,0.110454,...,0.607635,0.769401,0.805356,0.806033,0.824035,0.818250,0.818939,0.810804,0.818478,0.821633
3,178.902406,237.959759,296.593262,214.237673,242.034295,303.703468,308.347796,369.493873,362.386410,317.556128,...,185.160851,249.247097,305.348805,558.952004,532.089659,583.254714,387.462956,507.237644,568.566861,413.714900
4,2.591589,3.237982,3.713286,3.471541,0.234606,0.200418,0.171750,0.232800,0.328055,0.279325,...,0.201520,4.672811,5.872723,5.995486,6.429594,6.719794,7.869853,8.749872,9.463638,17.780395
5,0.223675,0.306863,0.302403,0.384304,4.770049,3.716620,3.406547,0.105045,0.165687,0.098417,...,4.035973,117.837300,154.013454,156.856839,164.395254,182.414403,220.149930,243.278295,176.483323,292.092323
6,137.658997,144.018704,129.503144,140.316276,82.041390,77.188933,74.303321,74.795613,71.817706,70.959082,...,75.847598,173.495098,152.852597,148.581624,144.923832,141.578143,148.027779,147.209341,121.201215,128.438125
7,257.066029,257.277432,257.653725,257.505181,256.072326,256.039748,256.029547,256.279257,256.329725,256.403301,...,256.062202,254.665335,254.440296,254.395904,254.425316,254.209042,253.957464,253.559709,254.078490,252.742171
8,255.693343,255.620497,255.517969,255.517570,257.851529,257.440097,257.307987,255.881086,255.889596,255.938329,...,257.706446,262.830393,264.139721,264.673620,264.686225,266.084360,267.923589,269.817363,264.072179,269.097300
9,1674.470344,1863.341474,2178.748660,1836.050487,2732.690115,3044.129653,3087.878565,2885.693794,2844.010720,2772.518665,...,2781.123747,1788.601158,2130.868731,3361.042080,3157.973145,3500.836655,2656.521356,3038.333079,3446.780292,2775.769078


In [19]:
best.to_csv('/home/mohit/Desktop/featuresUpdated/finalFeatures.csv')